In [1]:
df=spark.read.csv('./data/flights.csv',header=True,inferSchema=True)
from pyspark.sql.functions import monotonically_increasing_id 
df = df.select("*").withColumn("ID", monotonically_increasing_id())

In [2]:
# df.printSchema()

In [8]:
df=df.drop('TAXI_OUT','TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'YEAR', 
                       
                       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
                       'WEATHER_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
                       'FLIGHT_NUMBER', 'TAIL_NUMBER', 'AIR_TIME','AIR_SYSTEM_DELAY')
# drop na value
data=df.na.drop()


In [9]:
df.printSchema()

root
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = true)
 |-- ID: long (nullable = false)



In [10]:
#convert departure time to the number of minutes in a day
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
def convert(time):
    min=time%100
    h=int(time/100)
    return h*60+min

    
time_converter=udf(lambda s:convert(s))
data=data.select('*',time_converter(data['SCHEDULED_DEPARTURE']).alias('schedule_departure')).drop('SCHEDULED_DEPARTURE')

In [11]:
from pyspark.sql.types import IntegerType
from operator import add
#combine day and month:
months = [31,28,31,30,31,30,31,31,30,31,30,31]
accu=0
accu_months=[0]
for element in months:
    accu +=element
    accu_months.append(accu)
transform = udf(lambda month:accu_months[month-1],IntegerType())
data=data.select('*',(transform(data['MONTH'])+data['DAY']).alias('NEW_DAY'))
data=data.drop('MONTH')
data=data.drop('DAY')


In [13]:
data.printSchema()

root
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = true)
 |-- ID: long (nullable = false)
 |-- schedule_departure: string (nullable = true)
 |-- NEW_DAY: integer (nullable = true)

